# Python and Google Earth Engine


# To do! 
- [ ] Add learning goals!! 
- [ ] add something that sais run this in jpuyter! 
- [ ] add final analysis part
- [ ] create exercise




Many of you will know or have scrolled around in [google earth](https://earth.google.com/web/). Within Google Earth it is possible to expplore the earth behind your computer. Through the application a lot of different maps and information can be found, some straight forward such as the normal google maps base map, but also layers such as the age of the sea floor. Additionally, (or primarily if you ask geospatial people) it is a powerful cloud based computational platform where geospatial analysis can be carried out. All this information and all tools are gathered, and made available to everybody (with a google account...) in the "engine" behind Google Earth: Google Earth Engine (GEE). GEE was originally developed and made available as a _JavaScript API_. What does that exacly mean? Let's break it up into Javascript and API. 

### API
An API is a "Application Programming Interface", which is a standard language (or a set of rules) with wich pieces of software can commincate with eachother. API's are used for for example [accessing data](https://openweathermap.org/api/one-call-3#how) or to interact with the [command line tool: GDAL](https://geoscripting-wur.github.io/PythonRaster/) like we did yesterday. 


### JavaScript  
JavaScript (JS) is a widely used programming language, primarily for interacting with website. Javascript is an object oriented programming language, which means that it focuses on creating reusable code through objects. Objects can encapsulate both data (properties) and functionality (methods) in one single object. Object Oriented Programming (OOP) is a programing paradigm which is slightly different than how we have programmed so far. Python is also, or can also be used as, an OOP language. Working with the GEE Python API is a great oportunity to get familiair with object oriented programming while at the same time work with the extremely powerful engine from google. 

# Object Oriented Programming in Python
In this tutorial we will learn about how to interact with GEE. Before we do this we need to be able to think in a Object Oriented Programming way. Therefor we will need to understand how OOP is implemented in Python. 

In Python, objects are represented using classes. A class is a blueprint that defines the structure and behavior of objects. It encapsulates data (properties) and functions (methods) into a single object.

To create a class, we use the `class` keyword, followed by the class name. Here's an example of a simple class called `Person`:

In [ ]:
class Person:
    def __init__(self, name, age):
        self.name = name  # < this is a property
        self.age = age    # < this is also a property
    
    def greet(self):  # < This "function" is a method
        print(f"Hello, my name is {self.name} and I'm {self.age} years old.")

In the above code, the __init__ method is a special method called a *constructor*.  It is invoked automatically when an object is created from the class. The `self` parameter refers to the instance of the class and allows access to its properties and methods. In this example it means that every time self is passed to the class or a method in the class, this has access to all its own properties and methods. Generally, always when a method is created we give the self key word as one of the parameters. 

The Person class has two properties (name and age) and one method (greet). The greet method prints a greeting message with the person's name and age.

To create an instance of the Person class, you simply call the class as if it were a function and assign it to a variable:

In [ ]:
person1 = Person("Alice", 25)
person2 = Person("Bob", 30)

We created 2 _objects_, which are _instances_ of the _class_ _Person_, called `person1` and `person2`. We can now access the properties and call the methods of these objects:

```python
print(person1.name)  # Output: Alice
print(person2.age)   # Output: 30
person1.greet()      # Output: Hello, my name is Alice and I'm 25 years old.
person2.greet()      # Output: Hello, my name is Bob and I'm 30 years old.
```

This is a simple example, but you can imagine that these can get very complex. For example take a look at how a GeoPandas GeoSeries is created [here](https://github.com/geopandas/geopandas/blob/main/geopandas/geoseries.py). It is very complicated and hard to read, but very well documented so try to understand some of its functionality! The method [`to_json`](https://github.com/geopandas/geopandas/blob/main/geopandas/geoseries.py#L1209) contains very straight forward functionality (but it might still be hard to read).

One of the things that might occur to you is that the classes are defined slightly different than we have learned. In the geopandas example we see that a class is created using the following code: 

```python 
class GeoSeries(GeoPandasBase, Series):
    
```

While we learned we just had to call the following coode:

```python 
class GeoSeries:
```

The difference is that the code in the brackets is how "inheritance" is implemented. The class will _inherit_ all the functionality of the classes passed as arguments, in this case `GeoPandasBase` and `Series`. The `Series` object refers to the [Pandas.Series](https://github.com/pandas-dev/pandas/blob/main/pandas/core/series.py#L243C15-L243C15) object containing 6000 lines of code containing all sorts of functionality. In this way, the GeoPandas series contains all the functionality that is implemented in pandas *plus* all the functionality from GeoPandasBase *plus* all the functionality implemented in GeoPandas.Series itself.

Pfew, that's a lot of very complicated code and maybe a bit very overwelming and you do not need to understand all of it. The important thing to understand is the value of classes, objects and inheritance. How does this work in a simple example? 

So we established that with inheritance we can allow classes to take all the functionality from other classes and build on that. So let's create a new object called `Student`, which will inherit all properties and methods from `Person` which we created earlier. 


In [ ]:
class Student(Person):
    def __init__(self, name, age, student_id):
        super().__init__(name, age)
        self.student_id = student_id
    
    def study(self):
        print(f"{self.name} is studying.")



In the above code, the `Student` class inherits from the `Person` class (this class we will from now refer to as the superclass). It extends the functionality by adding a new property (student_id) and a new method (study). The `super()` function is used to call the superclass's __init__ method, allowing the subclass to initialize the inherited properties. 

The student class now contains methods and properties from Person as well as from Student: 

```python
student = Student("Eve", 22, "123456")
print(student.name)         # Output: Eve
print(student.student_id)   # Output: 123456
student.greet()             # Output: Hello, my name is Eve and I'm 22 years old.
student.study()             # Output: Eve is studying.
```

# Google Earth Engine

As already introduced, Google Earth Engine is a large collection of geospatial data with poerwful compitational capabilities. It allows users to analyze and visualize geospatial data on a global scale, making it a valuable tool for research about for example environmental monitoring and resources managent. 

One of the key features of GEE is its collection of raster datasources. Have a look at the [catalogue](https://developers.google.com/earth-engine/datasets/catalog) to see what is out there. The collection also contains raster time series and non-raster data although. For some use cases, check out [this article by Pérez-Cutillas and colleagues](https://www.sciencedirect.com/science/article/pii/S2352938522002154) from this year. They did a systematic review of research done using GEE. From their article:  

> The results of the meta-analysis following the systematic review showed that: (i) the Landsat 8 was the most widely-used satellite (25%); (i) the non-parametric classification methods, mainly Random Forest, were the most recurrent algorithms (31%); and (iii) the water resources assessment and prediction were the most common methodological applications (22%).
> 
> -- Pérez-Cutillas et al., 2023

## Night time light emision 
In this tutorial we will do a relatively straight forward analysis to show you some often used tools, but more importantly we want to show you how to work with GEE in python, how to find tools and documentation and how to read the (JS) documentation.

The analysis we will carry out is a comparison of how much light is emited in the Netherlands during the night through time. We will run you step by step through the procedure. The data we will use is a worldwide dataset from 2012 to now so feel free to carry out the analysis on a different area. 

The environment we will use will need the following dependencies. Save the following yaml in a new file called env.yaml and create and activate an environment: 

```yaml
name: pythonGee
channels:
  - conda-forge
dependencies:
  - python=3.10
  - jupyter
  - earthengine-api
  - geemap
  - pygadm
  - geopandas
  - geojson
```

To test try and import the following packages: 


In [2]:
import ee
import geemap 
import pygadm
import geopandas as gpd
import geojson

Google Earth engine is a google application, so not surprisingly you will need a google acount to be authorized using this account for working with their tools. The authentication will be done by a procedure started by calling the `Authenticate` function from `ee`. Follow these steps: 
1. Follow the url that will print after running the Authenticate function
1. require you to create a token by loging into your google account. 
1. Click Continue when you get the warning that google has not verified this app. This means that Google did not verify the code we are about to run, and it prevents you from your account being used by others. 
1. Next select all the tick boxes, this will allow you to use GEE data and it allows you to manage your data and permissions in the cloud storage, where the actual analysis will be run.
1. lastly you wil need to copy and pased the authorization code (and press enter) in the textbox which will apear after running the Authenticate method. 

In [ ]:
# ee.Authenticate()

You now have (hopefully) succesfully authenticated. Next we have to initialize using the `Initialize()` method: 

In [3]:
ee.Initialize()

If you get no further warnings or errors you have succesfullt authenticated and are ready to work with the earth engine library in python! Congrats! Next time you are working in the same environment, you will not have to authenticate again. You can just run the `Initialize` function and off you go! 

## Night time data

So we want to analyze how the light emitance has developed over the last 10 years. For this we will compare per province how much light was emited per month. 

For the analysis we will need data about emitted light. We will use the VIIRS Nighttime data. Read more about the data [here](https://developers.google.com/earth-engine/datasets/catalog/NOAA_VIIRS_DNB_MONTHLY_V1_VCMCFG). It is monthly average radiance composite images using nighttime data from the Visible Infrared Imaging Radiometer Suite (VIIRS) Day/Night Band (DNB). You can have a look at the data in [this esri viewer](https://www.arcgis.com/apps/mapviewer/index.html?layers=edabcbb5407547f5bc883018eb6e7986). To use this data we can create a `ImageCollection` object. Have a look at the [documentation](https://developers.google.com/earth-engine/apidocs/ee-imagecollection). We can see that for the implementation example we have 2 options. 1 is the JavaScript option and one is the Python Google Colab option. This is because Google is implementing all the JavaScript functionality also in python. Well actually, as far as we know all functionality seems to be implemented, however not all the documentation or example has been created for python. Luckily, if you switch between the JS and the Python documentation, we will see not much differences. Because we do not know at what point what python documentation will be available and because it is a good exercise to be able to read JavaScript documentation we will refer to the JavaScript documentation and point out important differences, but feel free to use the Python documentation when available. 

An image collection can be created using the following command:

In [4]:
# Import the night time light emision data.
viirs_image_collection = ee.ImageCollection("NOAA/VIIRS/DNB/MONTHLY_V1/VCMCFG")

You can see that this is the exact same code as in the JS documentation. However, they limit the collection to 3 images only, whereas we imported all images. After all we want to study the development over the entire timespan. An important difference between the JS API and the Python API is that to get information about an object we have to call the `getInfo()` method. So instead of in JS printing information as follows: 
```JavaScript
print('Image collection from a string', ee.ImageCollection("NOAA/VIIRS/DNB/MONTHLY_V1/VCMCFG"));
```

We can use the following command:

In [ ]:
print(type(ee.ImageCollection("NOAA/VIIRS/DNB/MONTHLY_V1/VCMCFG").getInfo()))

We need to call the `getInfo` method to convert the object information to a dictionary where we could extract useful information from. We can also show the ImageCollection object in jupyter by calling the object itself. 

In [ ]:
ee.ImageCollection("NOAA/VIIRS/DNB/MONTHLY_V1/VCMCFG")

We can see that the object has some properties. it has a `type`, `id`, `version`, an empty list as `bands`, 23 `properties` and 132 `features`. Note that these are very similair to what you would expect a [geojson](https://geojson.org/) containing a FeatureCollection to look like. 

The type, id and version are information from google about the data and the object type. At this point the object has an empty list as bands, which means at this points no bands. It has 23 properties. Note that these properties are properties from the collection, not from the individual images, so they contain information such as that it is a montly collection. Lastly the collection contains features: a list of 132 elements. Each element is an Image with again a `type`, `id`, `version`, `bands` and `properties`, but in this case no features. It is not a collection in the end. 

These features are the individual images, 1 per timestamp, so 11 years of 1 image per month. Let's look at one of these images. 

## GEE as a viewer! 

So as said before, Google Earth Engine is the engine behind Google Earth. It not only is a data collection, it also contains tools and a viewer. To interface with this viewer we will use another package called `geemap`. [Geemap](https://geemap.org/) is not only a viewer, it also contains very handy tools, for example to convert data to and from GEE native objects. More on that later.

To create amap we have to follow a view steps, which we will gracefully take from the [geemap getting started page](https://geemap.org/get-started/#add-earth-engine-data) and modify it a little for our use case.  

We will take the following steps: 
1. Initialize a map, with a starting position and a zoom leve. This map will stay where it is, and we will add layers to it as we go.
1. We will select 1 image from our image collection
1. We will add this image to the map 
1. We will also add a boundary of the netherlands to the same map. 

In [ ]:
# set our initial map parameters for Wageningen
center_lon = 51.962589
center_lat = 5.669627
zoomlevel = 8



# The following line will initialize the map
Map = geemap.Map(center=[center_lon, center_lat], zoom=zoomlevel, layer_ctrl=True)

# The following line will actually call and show the map:
Map


The map will stay and live in this window where it is. It is an interactive map with at this moment 1 standard basemap. To start over again you can call `Map.clear()`, that will remove all layers and other objects from the map and you will have to initialize it again. Step 1 done! Next step is to select one image from the image collection and to add it to the map. 

In [10]:
# Initial date of interest (inclusive).
from_date = '2017-01-01'
# Final date of interest (exclusive).
to_date = '2020-01-01'

image = viirs_image_collection\
            .filterDate(from_date, to_date)\
            .sort("system:time_start", True)\
            .first()

visualization_params = {
    'min': 0,
    'max': 10,
    'opacity': 0.5,
    'bands': ['avg_rad']
}

Map.addLayer(ee_object = image, 
             vis_params = visualization_params,
             name = 'My First Image')


In the map you can see that an image is added. But what did the code do? Try to understand it for yourself and look at the documentation if needed, but we will also explain here. 

In the first line a couple of things happened: 

```python 
image = viirs_image_collection.sort("system:time_start", True).first()
```

1. Firstly we sort the collection using a method of the imageCollection, see the [documentation here](https://developers.google.com/earth-engine/apidocs/ee-imagecollection-sort). It sorts the collection on a property, in this case that is `system:time_start`. Each image in the colllection has this property, it specifies the start date of the image (the values are an average over a period of time, from this timestamp to the `system:time_end` property). Lastly the boolean True refers to the argument `ascending`. In this case we want to sort the collection on `system:time_start` and we want to sort it ascending. It returns an image collection again.
1. Secondly, we select only the images between 2 dates that we defined, from 2017 to 2020. 
1. Lastly we call the method `First`. This is straight forward, the collection is now sorted as described above, and we want to return the first element of this collection. 

Next we define some visualization parameters we will use to put the image on the map. We select the range of images, we use an opacity and we want to show 1 band: `avg_rad`. If we look at the image we can see it has 2 bands: `avg_rad` and `cf_cvg`. See what they mean [here](https://developers.google.com/earth-engine/datasets/catalog/NOAA_VIIRS_DNB_MONTHLY_V1_VCMCFG#bands).

We then add the image to the map using the `AddLayer` method. It needs an earth engine object, visualization paramaters we defined earlier and a name. The last two are defaulted to nothing. 

If we look at the map we can see the image has indeed appeared. Great! Second and third step done! 

So for the last step we also want to add the boundaries of the netherlands. For getting the data we will use another package called `pygadm`. Pygadm is a dataset project from the Berkely University, and it's a great way to get world wide data on administrative boundaries on multiple scales. 

In [ ]:
netherlands = pygadm.get_items(admin='NLD', content_level=1)
ee_netherlands = geemap.geopandas_to_ee(netherlands)

Map.addLayer(ee_object = ee_netherlands, 
             name = 'Netherlands Boundaries')
Map.addLayerControl()

Using the `get_items` function, we can get the boundaries on content level 1, which are provinces in the Netherlands. We get this data in `GeoPandas GeoDataFrame` format, which we can not directly add to the map. However, we can use a handy conversion tool coming straight from the `geemap` package to convert it to an earth engine geometry. The object resulting from this we can put on the map quite easily. 

For more examples and tools, look at the [examples](https://github.com/gee-community/geemap/tree/master/examples/notebooks) supplied by geemap themselves! Especially note that one can [export a map to html](https://github.com/gee-community/geemap/blob/master/examples/notebooks/21_export_map_to_html_png.ipynb), that might come in useful during the project.

## Analysis in GEE

So far we have prepared well for visualization, we added the imagery for the entire world to the map. However, for the analysis we want to clip the imagery to the boundaries. After that we want to take the average of all the pixels per province. This will result in a single value per province per month. Schematically the steps will look like this:

1. From the ImageCollection, select the band we want to do the analysis on, filter the collection to the timerange we want to do the analysis and 
1. Collapse (is this the right word for this?) the image collection into a single image with many bands
1. Clip the images on the boundaries
1. Calculate the statistics per province
1. Export the data to from earth engine to an easier to work with format.

The first step can be done in one line. Several times we call a method on an image collection, which will result in an image collection. On this collection we will call another method and so fort. Look at the following line of code and try to understand what is happening before you read the explanation. Use the documentation! 

In [11]:
viirs_image = viirs_image_collection.select('avg_rad') \
                .filterDate(from_date, to_date) \
                .filterBounds(ee_netherlands) \
                .toBands() 

So 4 methods are called. In the entire line we go from an image collection from the whole world, over all available time steps. We filter it down to an image that has a footprint overlapping the netherlands, the  with 1 band per timestep. Why we collapse it into this single image will become clear in the next step. The separate methods are described below.

1. First, we select the band we are interested in, the `avg_rad` band, containing the avarage nighttime radiation per month. For this we use the [`select`](https://developers.google.com/earth-engine/apidocs/ee-imagecollection-select) method. 
1. We then select the images from the dates we are interested in using the [`filterDate`](https://developers.google.com/earth-engine/apidocs/ee-imagecollection-filterdate) method. 
1. On the resulting image collection we call the [`filterBounds`](https://developers.google.com/earth-engine/apidocs/ee-imagecollection-filterbounds) method. 
1. Finally we call the [`toBands`](https://developers.google.com/earth-engine/apidocs/ee-imagecollection-tobands) method. Note that this method results in an *`Image`*  instead of an *`ImageCollection`*. 

When you run the code above note that the code is run instantly, not matter the processing power of your laptop or virtual machine, while actually it is quite a large computation. How can this be? Actually the code and the calculation is not done on your computer, in fact there is no computing done at all yet for this call. This code creates an API Call, which will be sent to the Earth Engine cloud platform ones it needs to be called. For example when you now call the `viirs_iamge` variable, it takes longer. The API call is sent and the results are returned to your environment. 

In [ ]:
viirs_image

Next we will clip the image to the desired area, the boundary of the Nehterlands. This can be done by using the [`clip method`](https://developers.google.com/earth-engine/apidocs/ee-image-clip). We also could have added this to already long line of code above. Functionaly this has no difference. The decision on how to do this is primarily personal, every coder might do this difference. The most important aspect to this choice is to make it clear for others and for yourself what line is doing what and to define clear variable names. 

In [12]:
# Function to clip an image to the specified geometry
viirs_image_clipped = viirs_image.clip(ee_netherlands)

`viirs_image_clipped` now contains an image with 36 bands, verify this in your own environment. Do you understand the band names? 

We now want to calculate the average per province. We do this using a *reducer*, which we apply over an area. A reducer is an algorithm which will be applied over a set of values. The set of values in our case will be the pixel values within a province. How this works in earth engine, is that we use a method called [`reduceRegions`](https://developers.google.com/earth-engine/apidocs/ee-image-reduceregions). According to the documentation this method needs as input a collection and a reducer. As input we will give the polygons of the provinces in the Netherlands: `ee_netherlands`. As reducer we have options. To see what the options are, search for `reducer` in the filter in the documentation. Since we want the average over the collection, we will use the `ee.Reducer.mean()` function, which is not a reducer itself but it *returns* a reducer. That means we will need to actually call the function in the input! We also pass give the optional parameter scale, and we set it to 300 meters. Check the documentation to understand what this parameter does. 

In [ ]:
province_statistics = viirs_image_clipped.reduceRegions(
    collection = ee_netherlands,
    reducer = ee.Reducer.mean(),
    scale = 300
)

All right! We now have a featureCollection with an average radiation per province, per month over the time period. For analysis and visualization we now want to calculate the largest difference within the time period, and we want to classify this difference and show the results on the map. Now, this is probably all possible within earth engine, and if this is the easiest way for you, feel free to do so. Howeever, we learned to work with geopandas so it may be easier to use what we know. It is not a problem at all to export the output to a GeoPandas DataFrame and go from there, so let's do that! 

So the steps we will take are: 
1. export the statistics FeatureCollection to a GeoPandas DataFrame
1. Calculate the difference in radiation per province 
1. Classify this difference in 3 classes, small difference medium difference, large difference
1. Convert resulting DataFrame back into a Earth Engine object
1. Visualize the results

We start with exporting. `Geemap` comes with very handy tools for this. A [GeoPandas](https://geemap.org/common/?h=ee_to_geotiff#geemap.common.ee_to_geopandas) object is for us the most convenient format, but there are many other options. Check the documentation for other options! 


In [ ]:
df = geemap.ee_to_geopandas(province_statistics)

From here calculating the difference is straight forward, since we know (geo)pandas well. We now also have all the functionality that comes with GeoPandas, such as very easily writing the results to a geojson!

In [ ]:
df['diff'] = df['20170101_avg_rad'] - df['20191201_avg_rad']
df.to_file('test.geojson', drive='GeoJSON')

Now to see how the radiation behaved over time we have to do a little more work, since the dataframe in this format is not very useful for this. Try to read and understand the code below. 

In [1]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import matplotlib

In [ ]:
# Clean the dataframe a little
clean_df = df.drop(['GID_0', 'GID_1', 'ISO_1', 'HASC_1', 'NAME_0', 'NL_NAME_1', 'VARNAME_1', 'TYPE_1','CC_1', 'ENGTYPE_1'], axis=1)

# Reformat the dataframe so the rows depict measurements per timestamp per province, instead of an attribute per timestamp
melted_df = pd.melt(clean_df, id_vars=['NAME_1', 'geometry', 'diff'] , var_name='date', value_name='value')

# Convert the time string to a datetime format
melted_df['short_date'] = melted_df['date'].str[:8].str.strip()
melted_df['datetime'] = pd.to_datetime(melted_df['short_date'], format="%Y%m%d")


# Initialize the plot
fig, ax = plt.subplots(figsize=(9, 6))

# Create an empty list to save all lines in for the legend 
lines = []

# For each province, add a line with a different color 
for prov in melted_df['NAME_1'].unique():    
    
    # Create a dataframe for the province of interest
    d = melted_df[melted_df['NAME_1'] == prov]
    
    # Generate a random color
    random_rgb_values = list(np.random.random(3))
    random_color = matplotlib.colors.rgb2hex(random_rgb_values, keep_alpha=False)
    
    # Add a line for each province
    lines += ax.plot( d["datetime"].values, d["value"].values, color= random_color, lw=1, zorder=10, label=prov)
    
    
# Add the legend    
ax.legend(handles = lines)

We can see that there are some differences per province, but that there does not seem to a nationwide trend. Per province there might be. To find out, you can add a trendline to the graph, but this is out of scope for this tutorial. 

For the visualization we now want to categorize the differences in 3 classes, a little difference, medium difference and a large difference. To do this we will use a numpy function called `linspace` and a pandas function called `cut`. This results in an array with datatype `CategorizalClasses`, which earth engine does not understand so we convert it to a string. 


In [ ]:
breaks = np.linspace(df['diff'].min() - 0.0001, df['diff'].max(), 4)

# Assign values to classes
classes = ['Low', 'Mid', 'High']
df['class'] = pd.cut(df['diff'], bins=breaks, labels=classes, right=True)
df['class'] = df['class'].astype('str')

So the next step is to show the differences the earth engine map we made earlier. To do this, we have to convert the geopandas dataframe back to a earth engine object. Before we do this we have to tell geopandas what coordinate refrence system the geometries are in, apparently it was not exported from earth engine, but it does need it to be imported... 

In [ ]:
df = df.set_crs(4326)
new_ee_statistics = geemap.geopandas_to_ee(df)

In [ ]:
# Temp
new_ee_statistics = geemap.geojson_to_ee('ee_stats.json')

In [ ]:
# If you run the entire tutorial in one go, the map will show at the same location as it was first initialized.
# To show it hear, clear the map and create a new one. 
# If the map was not initizalized before, this line will throw an error. 
Map.clear()

Now we have the polygons back in the right format with the classes. To visualize we define a palette, which tells earth engine which color we will give to each class. For the legend we can use this dictionary directly. Of course we also want to add the latest night time imagery behind it and we want to add a layer control. 

In [16]:
Map = geemap.Map(zoomlevel=8)

palette = {
    'High': 'B81D13', # red
    'Mid': 'EFB700',  # yellow
    'Low': '008450'   # green
}


visualization_params = {
    'min': 0,
    'max': 10,
    'opacity': 0.7,
    'bands': ['20191201_avg_rad']
}

Map.addLayer(ee_object = viirs_image_clipped, 
             vis_params = visualization_params,
             name = 'Night Time')

Map.add_styled_vector(new_ee_statistics, column="class", palette=palette, layer_name="Styled vector")
Map.center_object(new_ee_statistics)

Map.add_legend(legend_title="Legend", legend_dict=palette, position = 'topright')

Map.addLayerControl()

Map